In [13]:
import requests
from requests import get
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
import numpy as np

from time import sleep
from random import randint
import unicodedata

In [16]:
pages = np.arange(422, 423, 1)


In [8]:
df=pd.DataFrame(columns=['nombre' ,'subj', 'col_1', 'col_2', 'anos'])
df

,nombre,subj,col_1,col_2,anos


In [17]:
for page in pages:
    response = requests.get("http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=" +str(page))
    html_doc = response.content
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    cadena=soup.find_all('td', {'class': 'textocurrienc'})[0].get_text(strip=True)
    sleep(randint(2,10))
    cadena= cadena.split(' ') 
    cadena= cadena[1:]

    
    nombre= ""
    for p in cadena:
        nombre= nombre+ p[0:].lower()
    nombre =nombre.split('(licencia)')
    nombre=nombre[0] 
    
    data = soup.find_all('td', {'class': ["TitulosVerde", 'textoNegro']})
    tabla= []
    for i in range(len(data)):
        tabla.append(data[i].get_text(strip=True))
    
    list_aux= []
    if 'Diversos Grupos Parlamentarios' in tabla:
        list_aux.append(tabla.index('Diversos Grupos Parlamentarios'))
    if 'Adherente' in tabla:
        list_aux.append(tabla.index('Adherente'))
    if 'De Grupo' in tabla:
        list_aux.append(tabla.index('De Grupo'))
    if 'Iniciante' in tabla:
        list_aux.append(tabla.index('Iniciante'))
    if 'Proponente' in tabla:
        list_aux.append(tabla.index('Proponente'))

    list_aux
    corta=min(list_aux)
    ficha_cur= tabla[0:corta]
    
    corte=[]
    for f in ficha_cur:
        if f.isupper() and len(f)>6:
            c=ficha_cur.index(f)
            corte.append(c)

    corte2=[]
    for f in ficha_cur:
        if f.isupper() and len(f)>6 and ficha_cur.index(f)!=0 :
            c=ficha_cur.index(f)-1
            corte2.append(c)        
    corte2.append(len(ficha_cur) -1)
        
    lista_sup= []    
    inicio=0
    ind=0
    
    for k,j in zip(corte,corte2):
        for f in ficha_cur[k:j+1]:
            if inicio in corte:
                inicio=inicio+1
            ind=inicio+2
            if ind <=len(ficha_cur) :
                if ficha_cur.index(f)==ind:
                    lista=[]
                    lista.append(nombre) 
                    lista.append(ficha_cur[k].replace(' ', '').lower())
                    #print(inicio, ind) 
                    for e in ficha_cur[inicio:ind+1]:
                        if ficha_cur.index(e) not in corte:
                            lista.append(e)
                        inicio=ind+1
                    lista_sup.append(lista)          
                    lista_ok=[]
                    for t in lista:
                        trans_tab = dict.fromkeys(map(ord, u'\u0301\u0308'), None)
                        t = unicodedata.normalize('NFKC', unicodedata.normalize('NFKD', t).translate(trans_tab)) 
                        lista_ok.append(t)   
                        if len(lista_ok)==5:
                            i=len(df)+1
                            df.loc[i]=lista_ok
                    

In [18]:
df.tail(20)

,nombre,subj,col_1,col_2,anos
2579,franciscojavierramireznavarrete,escolaridad,Licenciatura,Matematica Educativa,2006-2011
2580,franciscojavierramireznavarrete,actividadesdocentes,Docente,Asignatura: Matematicas,2013-2018
2581,mariachavezperez,escolaridad,Maestria,Gestion y Direccion de Instituciones,2016-2018
2582,mariachavezperez,escolaridad,Licenciatura,Educacion,1996-2000
2583,mariachavezperez,trayectoriapolitica,Candidata a Diputada Federal,MORENA,2015
2584,mariachavezperez,trayectoriapolitica,Consejera Nacional,MORENA,2015-2018
2585,agustingarciarubio,escolaridad,Licenciatura,Economia,1994-1999
2586,agustingarciarubio,escolaridad,Curso,Programacion de Computadoras,1993
2587,agustingarciarubio,escolaridad,Curso,"Computacion, Dominio de Internet, y las nuevas...",1998
2588,agustingarciarubio,administracionpublicafederal,Especialista en Campo,"Registro Federal de Electores Uruapan, Michoacan",2000


In [12]:
lista_sup

[['jorgeromeroherrera', 'escolaridad', 'Licenciatura', 'Abogado', '1997-2007'],
 ['jorgeromeroherrera',
  'escolaridad',
  'Dirigente Juvenil PAN-DF',
  'PAN',
  '2001-2003'],
 ['jorgeromeroherrera',
  'escolaridad',
  'Consejero Regional PAN-DF',
  'PAN',
  '2001-2007',
  'Integrante de la Comisión Permanente Nacional'],
 ['jorgeromeroherrera',
  'escolaridad',
  'PAN',
  '2012-2018',
  'Integrante de la Comisión Permanente Regional',
  'PAN'],
 ['jorgeromeroherrera',
  'trayectoriapolítica',
  '2012-2018',
  'Consejero Nacional',
  'PAN',
  '2012-2020'],
 ['jorgeromeroherrera',
  'trayectoriapolítica',
  'Jefe(a) Delegacional, Distrito Federal - Benito Juárez',
  'PAN',
  '2012-2015'],
 ['jorgeromeroherrera',
  'trayectoriapolítica',
  'Asambleísta Propietario, PAN',
  'IV Legislatura',
  '2006-2009'],
 ['jorgeromeroherrera',
  'trayectoriapolítica',
  'Asambleísta Propietario, PAN',
  'VII Legislatura',
  '2015-2018'],
 ['jorgeromeroherrera',
  'trayectoriapolítica',
  'Coordinador 

In [19]:
df.to_csv(r'C:\Users\Marlen\Documents\A.Marlen\A.datalab\ficha_cur_dip_500.csv', index=False,header=True )